## PCA based data distribution visualisation

In [4]:
from PIL import Image
import glob
import numpy

In [12]:
img_list = []
for filename in glob.glob('PIE/PIE/1/*.jpg'):
    im = Image.open(filename)
    arr = numpy.array(im)
    img_list.append(arr)
print(img_list)

[array([[36, 35, 35, ...,  0, 11,  7],
       [44, 45, 44, ...,  6, 14, 12],
       [49, 51, 51, ...,  8,  8,  5],
       ...,
       [36, 33, 29, ...,  7,  7, 12],
       [39, 35, 30, ...,  9, 10, 11],
       [40, 37, 33, ..., 11, 12,  9]], dtype=uint8), array([[ 91,  20,  25, ...,  28,  32,  45],
       [ 94,  16,  26, ...,  30,  33,  45],
       [ 98,  14,  22, ...,  33,  42,  56],
       ...,
       [ 70,  82,  87, ...,  46,  43,  41],
       [179, 188, 183, ...,  41,  37,  34],
       [249, 255, 231, ...,  37,  33,  29]], dtype=uint8), array([[ 88,  38,  16, ...,  56,  99, 162],
       [ 99,  45,  18, ..., 110, 141, 180],
       [100,  43,  11, ..., 161, 202, 196],
       ...,
       [  9,  15,  17, ..., 171, 165, 154],
       [ 28,  32,  33, ..., 154, 143, 131],
       [ 63,  65,  65, ..., 133, 118, 105]], dtype=uint8), array([[ 92,  42,  18, ...,  44,  70, 130],
       [ 97,  43,  16, ...,  86, 138, 104],
       [ 99,  43,  12, ..., 100, 177, 169],
       ...,
       [ 10,  17, 